## Soft margin SVM
Soft margin SVM is a branch of SVM (Support Vector Machines) that allows the model to make some level of misclassifications as to make the decision boundary (SOFTER)

Specifically, it aims to solve the following dual problem 

$$
max \space \sum_{i}\alpha_i - \frac{1}{2}\sum_i \sum_j y^{(i)}y^{(j)}a_ia_j<x^{(i)}, x^{(j)}> \\
s.t. \space C \ge \alpha_i \ge 0 , \sum_i y^{(i)}\alpha_i = 0
$$

With the following KKT conditions

$$
a_i = 0 \Rightarrow y^{(i)}(w^Tx^{(i)}+b) \ge 1 \\ 
a_i = C \Rightarrow y^{(i)}(w^Tx^{(i)}+b) \le 1 \\ 
C \ge a_i \ge 0 \Rightarrow y^{(i)}(w^Tx^{(i)}+b) = 1
$$

Along side with kernel trick, SMO is one of the powerful tools that can do so. 


## Implementation

### Required Values
- **point** corresponds to the training data $x_i$
- **target** corresponds to the training outputs $y_i$
- **C** is inversely proportional to the amount of mistakes we can afford. This depends on the scale of the problem. Mostly it’s set from $.01 \to100$
- **tol** is the amount of tolerance we will have for the KKT conditions.
- **prog_margin** is the padding we will employ for the calculation of $L$ and $H$ as to not make them equal. This will also serve as our margin in determining whether the two langrange multiplier has made any positive progress.
- **clip_padding** is the padding we will apply on the constraint $C \ge a_i \ge 0$ where we wil clip $a_i$ to either $C$ or $0$ if it’s within that padding

☝🏻 tol, prog_margin and clip_padding are mostly set to $1e^-3$ to $1e^-5$

In [65]:
import numpy as np
import math

np.random.seed(690420)

M: int = 100
D: int = 100
point = np.random.normal(size=(M, D), loc=0, scale=10).astype(np.float64)
target = np.random.choice([-1, 1], size=(M), replace=True)
c: np.float64 = .9
tol: np.float64 = 0.0001
prog_margin: np.float64 = 0.000001
clip_padding: np.float64 = 0.000000001
max_ch: int = 1000

### Kernel Function
Function responsible for the kernel trick

For gaussian Kernels we use the following 
$$
K(x ,z ) = exp(-\frac{||x-z||^2}{2\sigma})
$$

This can be sped up from the fact that $||x-z||^2 = x * x - 2x*z + z *z $

We can cache the dot product of vector to itself. We can also store the dot product of every 2 possible pair! but this may take a lot of memory

In [66]:
self_dot_cache: np.float32 = np.matmul(point, np.transpose(point))
sigma: int = 10

def kernel_gaussian(x: np.float32, z: np.array, cache_x: int=-1, cache_z: int=-1) -> float:
    '''
    cache_x and cache_z allows us to determine whether x and z are stored in self_dot_cache.
    This helps us calculate the dot product of a vector with itself easier
    '''
    dot_xx: float = self_dot_cache[cache_x, cache_x] if cache_x > -1 else np.dot(x, x)
    dot_zz: float = self_dot_cache[cache_z, cache_z] if cache_z > -1 else np.dot(z, z)
    dot_xz: float = self_dot_cache[cache_x, cache_z] if (cache_z > -1 and cache_x > -1) else np.dot(x, z)
    return math.e**((-1/(2 * sigma)) * (dot_xx - 2*dot_xz + dot_zz))

### Helper Functions

In [67]:


def compute_svm_err(x: int, alphs: np.float64, err_cache: np.float64, b: np.float64) -> np.float64:
    fx: np.float64 = obj_x(point[x], alphs, b, cache_val=x)
    err_cache[x] = fx - target[x]

    return err_cache[x]     

def obj_x(val: np.float64, alphs: np.float64, b: np.float64, cache_val: int=-1) -> np.float64:
    fx: np.float64 = -b
    alphs_zero: np.float64 = np.where(alphs != 0)[0]
    
    for i in alphs_zero:
        fx += alphs[i] *target[i] * kernel_gaussian(point[i], val, cache_x=i, cache_z=cache_val)
        
    return fx

def accuracy(alphs: np.float64, b: np.float64) -> np.float64:
    correct: int = 0
    for i in range(M):
        fx: np.float64 = obj_x(point[i], alphs, b, cache_val=i)
        h : int = 1 if fx >= 0 else -1
        correct += (target[i] == h)
    
    return correct / M


### Step Function
Lastly the function that takes a coordinate ascent step given the two multipliers

In [68]:
def take_step(i1: int, i2: int, alphs: np.float64, non_bound_alphs: np.int64, err_cache: np.float64, B: np.float64, log: bool=False) -> bool:
    print(f"Taking step for {i1} and {i2}")
    if (i1 == i2):
        print("Positions equal")
        return 0
    
    non_bound: bool = alphs[i1] != 0 and alphs[i1] != c
    b: np.float64 = B[0]

    E1: np.float64 = err_cache[i1] if non_bound else compute_svm_err(i1, alphs, err_cache, b)
    E2: np.float64 = err_cache[i2] 
    y1: int = target[i1]
    y2: int = target[i2]
    alph1: np.float64 = alphs[i1]
    alph2: np.float64 = alphs[i2]
    s: int = y1 * y2

    # Computation for L and H
    if (y1 == y2):
        L: np.float64 = max(0, alph1 + alph2 - c)
        H: np.float64 = min(c, alph1 + alph2)
    else:
        L: np.float64 = max(0, alph2 - alph1)
        H: np.float64 = min(c, c + alph2 - alph1)

    if (L == H):
        print("L and H equal")
        return 0
    
    K11: np.float64 = kernel_gaussian(point[i1], point[i1], cache_x=i1, cache_z=i1)
    K22: np.float64 = kernel_gaussian(point[i2], point[i2], cache_x=i2, cache_z=i2)
    K12: np.float64 = kernel_gaussian(point[i1], point[i2], cache_x=i1, cache_z=i2)
    eta: np.float64 = (2*K12)- K11 - K22

    if (eta < 0):
        alph2_new: np.float64 = alph2 - (y2*(E1 - E2)/eta)
        if (alph2_new < L):
            alph2_new = L
        elif (alph2_new > H):
            alph2_new = H
    else:
        f1: np.float64 = y1 * (E1 + B) - alph1 * K11 - s * alph2 * K12
        f2: np.float64 = y2 * (E2 + B) - s * alph1 * K12 - alph2 * K22
        L1: np.float64 = alph1 + s * (alph2 - L)
        H1: np.float64 = alph1 + s * (alph2 - H)
        Lobj: np.float64 = L1 * f1 + L * f2 + (0.5 * (L1 ** 2) * K11) + (0.5 * (L ** 2) * K22) + (s * L * L1 * K12)
        Hobj: np.float64 = H1 * f1 + H * f2 + (0.5 * (H1 ** 2) * K11) + (0.5 * (H ** 2) * K22) + (s * H * H1 * K12)

        if (Lobj < Hobj - prog_margin):
            alph2_new: np.float64 = H
        elif (Lobj > Hobj + prog_margin):
            alph2_new: np.float64 = L    
        else:
            alph2_new: np.float64 = alph2

    # clip
    if (alph2_new < clip_padding):
        alph2_new = 0
    elif (alph2_new > c - clip_padding):
        alph2_new = c
        
    if (abs(alph2_new - alph2) < (prog_margin * (alph2_new + alph2 + prog_margin))):
        print("Bad progress")
        return 0
    
    alph1_new: np.float64 = alph1 + (s*(alph2 - alph2_new))

    # update tresholds
    b1: np.float64 = E1 + y1*K11*(alph1_new - alph1) + y2*K12*(alph2_new - alph2) + b
    b2: np.float64 = E2 + y1*K12*(alph1_new - alph1) + y2*K22*(alph2_new - alph2) + b
    B[0] = (b1/2) + (b2/2)
    
    # update err_cache
    err_cache[i1], err_cache[i2] = 0, 0
    for i in non_bound_alphs:
        if (i == i1 or i == i2):
            continue
        
        K1k: np.float64 = kernel_gaussian(point[i], point[i1], cache_x=i, cache_z=i1)
        K2k: np.float64 = kernel_gaussian(point[i], point[i2], cache_x=i, cache_z=i2)

        err_cache[i] = err_cache[i] + y1*K1k*(alph1_new - alph1) + y2*K2k*(alph2_new - alph2) + b - B[0]
    
    # update alphs
    alphs[i1], alphs[i2] = alph1_new, alph2_new
    
    if (log):
        print(f"======= Step successful for {i1} and {i2} =======")
        print(f"a1: {alph1} -> {alph1_new} | a2: {alph2} -> {alph2_new}")
        print(f"b: {b} -> {B[0]}")
        print(f"err_cache: {err_cache}")
        print("==================================================")

    return 1

### Examine Alpha Function
The second function which is responsible for checking the first langrange multiplier that is chosen and responsible for picking the next langrange multiplier

In [69]:
def examine_a(i2: int, alphs: np.array, non_bound: bool, non_bound_alphs: np.int64, err_cache: np.array, B: np.float64) -> bool:
    non_b_len: int = len(non_bound_alphs)
    b: np.float64 = B[0]
    E2: np.float64 = err_cache[i2] if non_bound else compute_svm_err(i2, alphs, err_cache, b)
    r2: np.float64 = E2 * target[i2]
    alph2: np.float64 = alphs[i2]

    if ((r2 < -tol and alph2 < c) or (r2 > tol and alph2 > 0)):
        print("choosing second multipier")
        if (non_b_len > 1):
            # Second heuristic using optimal err for step estimation
            positive: bool = (err_cache[i2] > 0)
            i1: int = i2

            for i in non_bound_alphs:
                if (i == i2):
                    continue

                if (i1 == i2):
                    i1 = i
                elif (positive and err_cache[i] < err_cache[i1]):
                    i1 = i
                elif (not positive and err_cache[i] > err_cache[i1]):
                    i1 = i

            if (take_step(i1, i2, alphs, non_bound_alphs, err_cache, B)):
                return 1

        # take non-bound alps
        if non_b_len > 0:
            start: int = np.random.randint(size=(1), low=0, high=non_b_len)[0]
            for i in range(non_b_len):
                pos: int = (start + i)%non_b_len
                if (take_step(non_bound_alphs[pos], i2, alphs, non_bound_alphs, err_cache, B)):
                    return 1
        
        # loop through entire training set
        start: int = np.random.randint(size=(1), low=0, high=M)[0]
        for i in range(M):
            if (take_step((start + i)%M, i2, alphs, non_bound_alphs, err_cache, B)):
                return 1
    
    print("already satisfy kkt conditions")
    return 0

### Train Function
The train function which is responsible for picking the first langrange multiplier from a set of langrange multipliers. It is also responsible for initializing the important variables

In [70]:
def smo_train() -> dict:
    alphs: np.float64 = np.zeros(shape=(M), dtype=np.float64)
    err_cache: np.float64 = np.zeros(shape=(M), dtype=np.float64)
    B: np.float64 = np.array([0.], dtype=np.float64)

    examine_all: bool = True
    num_changed: int = 0
    total_changed: int = 0

    while num_changed > 0 or examine_all:
        print("choosing first multiplier")
        num_changed = 0

        non_bound_alphs = np.where((alphs != 0) & (alphs != c))[0]

        if examine_all:
            for i in range(M):
                num_changed += examine_a(i, alphs, False, non_bound_alphs, err_cache, B)
        else:
            for i in non_bound_alphs:
                num_changed += examine_a(i, alphs, True, non_bound_alphs, err_cache, B)

        if examine_all:
            examine_all = False
        elif num_changed == 0:
            examine_all = True

        total_changed += num_changed
        if total_changed > max_ch:
            return {"alphs": alphs, "err_cache": err_cache, "b": B[0]}

        print("total iterations: ", total_changed)

    print(err_cache)
    return {"alphs": alphs, "err_cache": err_cache, "b": B[0]}

### Test

In [71]:
import matplotlib.pyplot as plt

'''
x = [(0.1 + (i * 0.1)) for i in range(1, 10)]
y = []

for i in range(len(x)):
    c = x[i]
    res = smo_train()
    y.append(accuracy(res["alphs"], res["b"]))
    
plt.xlabel("c")
plt.ylabel("average distance")
plt.scatter(x, y)
'''

res = smo_train()
print(accuracy(res["alphs"], res["b"]))

choosing first multiplier
choosing second multipier
Taking step for 3 and 0
L and H equal
Taking step for 4 and 0
choosing second multipier
Taking step for 3 and 1
choosing second multipier
Taking step for 54 and 2
L and H equal
Taking step for 55 and 2
already satisfy kkt conditions
already satisfy kkt conditions
choosing second multipier
Taking step for 76 and 5
L and H equal
Taking step for 77 and 5
L and H equal
Taking step for 78 and 5
L and H equal
Taking step for 79 and 5
choosing second multipier
Taking step for 92 and 6
choosing second multipier
Taking step for 13 and 7
choosing second multipier
Taking step for 76 and 8
choosing second multipier
Taking step for 59 and 9
L and H equal
Taking step for 60 and 9
L and H equal
Taking step for 61 and 9
L and H equal
Taking step for 62 and 9
L and H equal
Taking step for 63 and 9
choosing second multipier
Taking step for 3 and 10
choosing second multipier
Taking step for 23 and 11
L and H equal
Taking step for 24 and 11
L and H equal

In [72]:
#plt.plot(x, [abs(i) for i in y])
'''
alphs: np.float64 = np.zeros(shape=(M), dtype=np.float64)
print(accuracy(alphs, 0))
print(res["b"])
'''

'\nalphs: np.float64 = np.zeros(shape=(M), dtype=np.float64)\nprint(accuracy(alphs, 0))\nprint(res["b"])\n'